In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from collections import Counter


## Read Data

In [3]:
df = pd.read_csv('Data/indexProcessed.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'Data/indexProcessed.csv'

In [3]:
# Read indexProcessed.csv
df = pd.read_csv('Data/indexProcessed.csv')
df.dtypes

Index         object
Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
CloseUSD     float64
dtype: object

In [4]:
# Read indexInfo.csv
df_ref = pd.read_csv('Data/indexInfo.csv')
df_ref.dtypes

Region      object
Exchange    object
Index       object
Currency    object
dtype: object

In [5]:
# Merged DF 
Merged_df = pd.merge(df_ref,df, how='inner')
Merged_df

,Region,Exchange,Index,Currency,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,United States,New York Stock Exchange,NYA,USD,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0,528.690002
1,United States,New York Stock Exchange,NYA,USD,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0,527.210022
2,United States,New York Stock Exchange,NYA,USD,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0,527.840027
3,United States,New York Stock Exchange,NYA,USD,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0,531.119995
4,United States,New York Stock Exchange,NYA,USD,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0,532.070007
...,...,...,...,...,...,...,...,...,...,...,...,...
104219,South Africa,Johannesburg Stock Exchange,J203.JO,ZAR,2021-05-25,66054.921880,66812.453130,66022.976560,66076.679690,66076.679690,0.0,4625.367578
104220,South Africa,Johannesburg Stock Exchange,J203.JO,ZAR,2021-05-26,66076.679690,66446.367190,66030.351560,66108.226560,66108.226560,0.0,4627.575859
104221,South Africa,Johannesburg Stock Exchange,J203.JO,ZAR,2021-05-27,66108.226560,66940.250000,66102.546880,66940.250000,66940.250000,0.0,4685.817500
104222,South Africa,Johannesburg Stock Exchange,J203.JO,ZAR,2021-05-28,66940.250000,67726.562500,66794.609380,67554.859380,67554.859380,0.0,4728.840157


In [6]:
# Dataframe for NYA

df_NYA = Merged_df[Merged_df['Index'] == 'NYA']
df_NYA

,Region,Exchange,Index,Currency,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,United States,New York Stock Exchange,NYA,USD,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.000000e+00,528.690002
1,United States,New York Stock Exchange,NYA,USD,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.000000e+00,527.210022
2,United States,New York Stock Exchange,NYA,USD,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.000000e+00,527.840027
3,United States,New York Stock Exchange,NYA,USD,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.000000e+00,531.119995
4,United States,New York Stock Exchange,NYA,USD,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.000000e+00,532.070007
...,...,...,...,...,...,...,...,...,...,...,...,...
13942,United States,New York Stock Exchange,NYA,USD,2021-05-24,16375.000000,16508.519530,16375.000000,16464.689450,16464.689450,2.947400e+09,16464.689450
13943,United States,New York Stock Exchange,NYA,USD,2021-05-25,16464.689450,16525.810550,16375.150390,16390.189450,16390.189450,3.420870e+09,16390.189450
13944,United States,New York Stock Exchange,NYA,USD,2021-05-26,16390.189450,16466.339840,16388.320310,16451.960940,16451.960940,3.674490e+09,16451.960940
13945,United States,New York Stock Exchange,NYA,USD,2021-05-27,16451.960940,16546.359380,16451.960940,16531.949220,16531.949220,5.201110e+09,16531.949220


## Create Target 

In [7]:
# Find delta of today's close price from yesterday's close price 

df_NYA['Delta'] = df_NYA['Close'].diff()
df_NYA

C:\Users\dhava\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Region,Exchange,Index,Currency,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD,Delta
0,United States,New York Stock Exchange,NYA,USD,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.000000e+00,528.690002,NaN
1,United States,New York Stock Exchange,NYA,USD,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.000000e+00,527.210022,-1.479980
2,United States,New York Stock Exchange,NYA,USD,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.000000e+00,527.840027,0.630005
3,United States,New York Stock Exchange,NYA,USD,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.000000e+00,531.119995,3.279968
4,United States,New York Stock Exchange,NYA,USD,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.000000e+00,532.070007,0.950012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13942,United States,New York Stock Exchange,NYA,USD,2021-05-24,16375.000000,16508.519530,16375.000000,16464.689450,16464.689450,2.947400e+09,16464.689450,89.689450
13943,United States,New York Stock Exchange,NYA,USD,2021-05-25,16464.689450,16525.810550,16375.150390,16390.189450,16390.189450,3.420870e+09,16390.189450,-74.500000
13944,United States,New York Stock Exchange,NYA,USD,2021-05-26,16390.189450,16466.339840,16388.320310,16451.960940,16451.960940,3.674490e+09,16451.960940,61.771490
13945,United States,New York Stock Exchange,NYA,USD,2021-05-27,16451.960940,16546.359380,16451.960940,16531.949220,16531.949220,5.201110e+09,16531.949220,79.988280


In [8]:
#  Add traget column to dataframe

df_NYA['Target'] = np.where(df_NYA['Delta']>=0, 'Up', 'Down')
df_NYA

C:\Users\dhava\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Region,Exchange,Index,Currency,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD,Delta,Target
0,United States,New York Stock Exchange,NYA,USD,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.000000e+00,528.690002,NaN,Down
1,United States,New York Stock Exchange,NYA,USD,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.000000e+00,527.210022,-1.479980,Down
2,United States,New York Stock Exchange,NYA,USD,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.000000e+00,527.840027,0.630005,Up
3,United States,New York Stock Exchange,NYA,USD,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.000000e+00,531.119995,3.279968,Up
4,United States,New York Stock Exchange,NYA,USD,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.000000e+00,532.070007,0.950012,Up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13942,United States,New York Stock Exchange,NYA,USD,2021-05-24,16375.000000,16508.519530,16375.000000,16464.689450,16464.689450,2.947400e+09,16464.689450,89.689450,Up
13943,United States,New York Stock Exchange,NYA,USD,2021-05-25,16464.689450,16525.810550,16375.150390,16390.189450,16390.189450,3.420870e+09,16390.189450,-74.500000,Down
13944,United States,New York Stock Exchange,NYA,USD,2021-05-26,16390.189450,16466.339840,16388.320310,16451.960940,16451.960940,3.674490e+09,16451.960940,61.771490,Up
13945,United States,New York Stock Exchange,NYA,USD,2021-05-27,16451.960940,16546.359380,16451.960940,16531.949220,16531.949220,5.201110e+09,16531.949220,79.988280,Up


In [9]:
df_NYA = df_NYA.dropna(how="any")
df_NYA

,Region,Exchange,Index,Currency,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD,Delta,Target
1,United States,New York Stock Exchange,NYA,USD,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.000000e+00,527.210022,-1.479980,Down
2,United States,New York Stock Exchange,NYA,USD,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.000000e+00,527.840027,0.630005,Up
3,United States,New York Stock Exchange,NYA,USD,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.000000e+00,531.119995,3.279968,Up
4,United States,New York Stock Exchange,NYA,USD,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.000000e+00,532.070007,0.950012,Up
5,United States,New York Stock Exchange,NYA,USD,1966-01-07,532.599976,532.599976,532.599976,532.599976,532.599976,0.000000e+00,532.599976,0.529969,Up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13942,United States,New York Stock Exchange,NYA,USD,2021-05-24,16375.000000,16508.519530,16375.000000,16464.689450,16464.689450,2.947400e+09,16464.689450,89.689450,Up
13943,United States,New York Stock Exchange,NYA,USD,2021-05-25,16464.689450,16525.810550,16375.150390,16390.189450,16390.189450,3.420870e+09,16390.189450,-74.500000,Down
13944,United States,New York Stock Exchange,NYA,USD,2021-05-26,16390.189450,16466.339840,16388.320310,16451.960940,16451.960940,3.674490e+09,16451.960940,61.771490,Up
13945,United States,New York Stock Exchange,NYA,USD,2021-05-27,16451.960940,16546.359380,16451.960940,16531.949220,16531.949220,5.201110e+09,16531.949220,79.988280,Up


In [10]:
list(df_NYA)

['Region',
 'Exchange',
 'Index',
 'Currency',
 'Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'CloseUSD',
 'Delta',
 'Target']

Below we are creating a baseline model

# LogisticRegression 

In [11]:
from sklearn.linear_model import LogisticRegression

y = df_NYA['Target']
X = df_NYA[['Open',
 'High',
 'Low',
 'Close',
 'Adj Close'
           ]]

clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X)

clf.predict_proba(X)


clf.score(X, y)


0.65481141545963

# Split the Data into Training and Testing

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, = train_test_split (X,
                                                     y,
                                                     random_state =1,
                                                     stratify = y)

X_train.shape
Counter(y_train)

Counter({'Up': 5580, 'Down': 4879})

# Balanced Random Forest Classifier

In [81]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Create a random forest classifier.
model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 

# Fitting the model
model = model.fit(X_train, y_train)

In [82]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.614647150570679

In [83]:
# Display the confusion matrix
#from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, y_pred)

array([[ 981,  646],
       [ 695, 1165]], dtype=int64)

In [84]:
# Print the imbalanced classification report
#from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

       Down       0.59      0.60      0.63      0.59      0.61      0.38      1627
         Up       0.64      0.63      0.60      0.63      0.61      0.38      1860

avg / total       0.62      0.62      0.61      0.62      0.61      0.38      3487



In [85]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_,X.columns), reverse=True)

[(0.24145937331501943, 'Open'),
 (0.19802548934441092, 'Adj Close'),
 (0.19659097226020045, 'Close'),
 (0.1840205920005052, 'High'),
 (0.1799035730798641, 'Low')]

# SMOTE Oversampling

In [86]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample( X_train, y_train)



In [87]:
Counter(y_resampled)

Counter({'Up': 5580, 'Down': 5580})

In [88]:
# Train the Logistic Regression model using the resampled data
#from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [89]:
# Calculated the balanced accuracy score
#from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.6430439624349849

In [90]:
# Display the confusion matrix
#from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, y_pred)

array([[1572,   55],
       [1265,  595]], dtype=int64)

In [91]:
# Print the imbalanced classification report
#from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

       Down       0.55      0.97      0.32      0.70      0.56      0.33      1627
         Up       0.92      0.32      0.97      0.47      0.56      0.29      1860

avg / total       0.75      0.62      0.66      0.58      0.56      0.31      3487



# Undersampling

In [92]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'Down': 4879, 'Up': 4879})

In [93]:
# Train the Logistic Regression model using the resampled data
#from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [94]:
# Calculated the balanced accuracy score
#from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.6428136421013674

In [95]:
# Display the confusion matrix
#from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, y_pred)

array([[1573,   54],
       [1267,  593]], dtype=int64)

In [96]:
# Print the imbalanced classification report
#from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

       Down       0.55      0.97      0.32      0.70      0.56      0.33      1627
         Up       0.92      0.32      0.97      0.47      0.56      0.29      1860

avg / total       0.75      0.62      0.66      0.58      0.56      0.31      3487

